# Tearing the Mask off Python Decorators
## Deep look into some advanced concepts
<img src='images/jelly.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@infonautica?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Leonid Danilov</a>
        on 
        <a href='https://www.pexels.com/photo/photo-of-jellyfish-2690765/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Introduction

### Functions Are Objects

One of the many things you will love about Python is its ability to represent anything as objects and functions are no exception. For people who is first reading this, passing a function as an argument to another function may seem strange but is completely legal to do so:

In [23]:
def my_func(arg=42):
    """
    Placeholder function
    """
    print('Printing the function\'s argument')
print(my_func)

<function my_func at 0x000001BF0530CF70>


As objects go, functions are absolutely the same as:
- str
- int, float
- pandas.DataFrame
- list, tuple, dict
- modules: os, datatime, numpy

You may assign functions to a new variable and use it to call the function:

In [24]:
new_func = my_func
new_func()

Printing the function's argument


Now this variable also contains the function's attributes:

In [25]:
# Get the docstring of a function
new_func.__doc__

'\n    Placeholder function\n    '

In [26]:
# Get function name
new_func.__name__

'my_func'

In [27]:
# Default argument values
new_func.__defaults__

(42,)

You can also store each function in other objects such as lists, dictionaries and call them:

In [28]:
funcs = [str.lower, print, range, str.startswith]
for func in funcs:
    print(f'The function name is \'{func.__name__}\'')

The function name is 'lower'
The function name is 'print'
The function name is 'range'
The function name is 'startswith'


In [29]:
func_dict = {
    'lower': str.lower,
    'print': print,
    'range': range,
    'startswith': str.startswith
}

func_dict['print'](func_dict['lower']('PYTHON'))

python


> Important note to be useful in later sections: Using the function with parentheses, `my_func()`, is called __'calling'__ the function while writing without, `my_func`, is called __referencing__. As you have seen, `print(my_func)` prints the function's
index in memory.

### Scope

Consider this conversation between Bob and Job:
- Bob: 'Jon, why did not you come to the lesson yesterday?'
- Jon: 'I had a flu...'

When Bob asks the reason of Jon's absence in yesterday's class, we know he is referring to the Jon standing next to him not some random Jon in another country. As humans, it is not difficult to notice this but programming languages use something called scope to tell which name we are referring to in our programs.

In Python, names can be variables, function names, modules name, etc. Consider these two variables:

In [30]:
a = 24
b = 42
print(a)

24


Here, `print` had no trouble to tell that we are referring to the `a` we just defined. Now consider this:

In [31]:
def foo():
    a = 100
    print(a)

What do you think will happen if we run `foo`? Will it print 24 or 100?

In [32]:
foo()

100


How did Python differentiate between the `a` we defined in the beginning or in the function? This is where scope gets interesting, because we are introducing layers of scope:

<img src='images/1.png'></img>

The above image shows the scope for this little script:

In [33]:
a = 24
b = 42
print(a)

def foo():
    a = 100
    print(a)
    
foo()

24
100


The global scope is the overall scope of your script/program. Variables, functions, etc. with the same indentation level as `a` and `b` in the beginning will be in the global scope. For example, `foo` is in the global scope but its variable `a` is in the scope that is local to `foo`. 

In one global scope, there can be many local scopes. Each temporary variables in `for` loops and list comprehensions, return values of context managers will be local inside their code block and cannot be accessed from the global scope.

Here, we add a for loop at the end of our little script:

In [34]:
for num in range(10):
    print(num)  # num is local to this for loop

0
1
2
3
4
5
6
7
8
9


So, a rule of thumb is that Python interpreter will not be able to access a name that is in one level inner of the current scope. There is also a bigger level of scope outside `global`:

<img src='images/2.png'></img>

Built-in scope contains all the modules and packages that came with Python's installation and the ones that were installed with `pip` or `conda`. 

Now, let's explore another case. In our `foo` function, we want to modify the value of global `a`. We want it to be a string now but if we write `a = 'some text'` inside the function, Python will just create a new variable without modify the global `a`. 

Python provides us with a keyword that lets us specify we are referring to names in the `global` scope:

In [35]:
# Before
def foo():
    a = 'some text'
    print(a)

foo()
print(a)

some text
24


In [36]:
# Using `global` keyword
# Before
def foo():
    global a
    a = 'some text'
    print(a)
foo()
print(a)

some text
some text


Writing `global 'name'` will let us modify the values of names in the `global` scope. 

BTW, bad news, I left out one level of scope in the above image. Between `global` and `local`, there is one level we did not cover:

<img src='images/3.png'></img>

`nonlocal` scope comes in to the play when we have, for example, nested functions:

In [61]:
def outer():
    # Create a dummy variable
    my_var = 'Python'
    
    def inner():
        # Try to change the value of the dummy
        my_var = 'Data Science'
        print(my_var)
    # Call the inner function which tries to modify `my_var`
    inner()
    # Check if successful
    print(my_var)

outer()

Data Science
Python


In nested function `outer`, we first create a variable called `my_var` and assign it to the string `Python`. Then we decide to create a new `inner` function and want to assign `my_var` a new value, `Data Science` and print it. But if we run it, we see that `my_var` is still assigned to 'Python'. We cannot use `global` keyword since `my_var` is not in the global scope. 

For such cases, you can use `nonlocal` keyword which gives access to all the names in the scope of the outer function (nonlocal) but not the `global`:

In [62]:
def outer():
    # Create a dummy variable
    my_var = 'Python'
    
    def inner():
        # Try to change the value of the dummy with nonlocal
        nonlocal my_var
        my_var = 'Data Science'
        print(my_var)
    # Call the inner function which tries to modify `my_var`
    inner()
    # Check if successful
    print(my_var)

outer()

Data Science
Data Science


In conclusion, scope tells the Python interpreter where to look for names in our program. There can be four levels of scope in a single script/program:
- Built-in: all the package names installed with Python and `pip`
- Global: general scope, all names that has no indentation in the script
- Local: contains local variables in each code block such as functions, loops, list comprehensions, etc.
- Nonlocal: an extra level of scope between `global` and `local` in the case of nested functions

### Closure

### Finally, Decorators